In [2]:
import numpy as np
from statsmodels.stats.weightstats import ztest, ttest_ind
from scipy.stats import chi2_contingency, fisher_exact
from scipy.stats import norm, expon, t, hypergeom, chi2
from scipy import mean, std
import matplotlib.pyplot as plt

In [3]:
def choice(ps):
    return np.random.choice(len(ps), p=ps)

def simulate_abtest(funnels, N):
    traffic_split = [x[1] for x in funnels]
    observations = np.zeros([len(funnels), len(funnels[0][0])])
    for _ in range(N):
        which_funnel = choice(traffic_split)
        funnel_outcome = choice(funnels[which_funnel][0])
        observations[which_funnel][funnel_outcome] += 1
    return observations

In [22]:
funnels = [
    [[0.80, 0.20], 0.6], # the first vector is the actual outcomes,
    [[0.80, 0.20], 0.4], # the second is the traffic split
]
N = 10*1000

observations = simulate_abtest(funnels, N)
raw_data = int(observations[0][0]) * [1] + int(observations[0][1]) * [0], int(observations[1][0]) * [1] + int(observations[1][1]) * [0]
print('Observations:\n', observations)
ch = chi2_contingency(observations, correction=False)
print('Chi-sq p = %.3f' % ch[1])
zt = ztest(*raw_data)
print('Z-test p = %.3f' % zt[1])
tt = ttest_ind(*raw_data)
print('t-test p = %.3f' % zt[1])

Observations:
 [[4825. 1183.]
 [3211.  781.]]
Chi-sq p = 0.876
Z-test p = 0.876
t-test p = 0.876


In [26]:
funnels = [
    [[0.80, 0.10, 0.10], 0.6], # the first vector is the actual outcomes,
    [[0.80, 0.10, 0.10], 0.2], # the second is the traffic split
    [[0.79, 0.11, 0.10], 0.1],
    [[0.70, 0.20, 0.10], 0.1],
]
N = 10*1000

observations = simulate_abtest(funnels, N)
print('Observations:\n', observations)
ch = chi2_contingency(observations, correction=False)
print('Chi-sq p = %.3f' % ch[1])

Observations:
 [[4748.  595.  573.]
 [1657.  197.  231.]
 [ 807.   98.  103.]
 [ 710.  195.   86.]]
Chi-sq p = 0.000


In [51]:
def chi_squared(observations):
    row_marginals = np.sum(observations, axis=1)
    col_marginals = np.sum(observations, axis=0)
    N = np.sum(observations)
    chisq = 0
    for i in range(len(row_marginals)):
        for j in range(len(col_marginals)):
            expected = row_marginals[i] * col_marginals[j] / N
            chisq += (observations[i][j] - expected)**2 / expected
    dof = (len(row_marginals) - 1) * (len(col_marginals) - 1)
    p_value = 1.0 - chi2(dof).cdf(chisq)
    return (chisq, p_value)

In [54]:
funnels = [
    [[0.80, 0.10, 0.10], 0.6], # the first vector is the actual outcomes,
    [[0.80, 0.10, 0.10], 0.2], # the second is the traffic split
    [[0.80, 0.10, 0.10], 0.1],
    [[0.80, 0.10, 0.10], 0.1],
]
N = 10*1000

observations = simulate_abtest(funnels, N)
print('Observations:\n', observations)
ch_scipy = chi2_contingency(observations, correction=False)
print('Statsmodel chi-sq test statistic = %.3f' % ch_scipy[0])
print('Statsmodel chi-sq p = %.3f' % ch_scipy[1])
ch_our = chi_squared(observations)
print('Our chi-sq test statistic = %.3f' % ch_our[0])
print('Our chi-sq p = %.3f' % ch_our[1])

Observations:
 [[4846.  594.  591.]
 [1628.  188.  171.]
 [ 767.  100.   98.]
 [ 824.   84.  109.]]
Statsmodel chi-sq test statistic = 7.324
Statsmodel chi-sq p = 0.292
Our chi-sq test statistic = 7.324
Our chi-sq p = 0.292


In [56]:
funnels = [
    [[0.80, 0.20], 0.6], # the first vector is the actual outcomes,
    [[0.80, 0.20], 0.4], # the second is the traffic split
]
N = 10*1000

observations = simulate_abtest(funnels, N)
raw_data = int(observations[0][0]) * [1] + int(observations[0][1]) * [0], int(observations[1][0]) * [1] + int(observations[1][1]) * [0]
print('Observations:\n', observations)
ch = chi2_contingency(observations, correction=False)
print('Chi-sq test statistic = %.3f' % ch[0])
print('Chi-sq p = %.3f' % ch[1])
zt = ztest(*raw_data)
print('Z-test z = %.3f' % zt[0])
print('Z-test z^2 = %.3f' % zt[0]**2)
print('Z-test p = %.3f' % zt[1])

Observations:
 [[4836. 1193.]
 [3147.  824.]]
Chi-sq test statistic = 1.378
Chi-sq p = 0.240
Z-test z = 1.174
Z-test z^2 = 1.378
Z-test p = 0.240


In [9]:
funnels = [
    [[0.80, 0.20], 0.6], # the first vector is the actual outcomes,
    [[0.80, 0.20], 0.4], # the second is the traffic split
]
N = 10*1000

observations = simulate_abtest(funnels, N)
raw_data = int(observations[0][0]) * [1] + int(observations[0][1]) * [0], int(observations[1][0]) * [1] + int(observations[1][1]) * [0]
print('Observations:\n', observations)
ch = chi2_contingency(observations, correction=False)
print('Chi-sq p = %.3f' % ch[1])
zt = ztest(*raw_data, alternative='two-sided')
print('Z-test p (TWO TAILED) = %.3f' % zt[1])
tt = ttest_ind(*raw_data, alternative='two-sided')
print('t-test p (TWO TAILED) = %.3f' % zt[1])
zt = ztest(*raw_data, alternative='larger')
print('Z-test p (ONE TAILED) = %.3f' % zt[1])
tt = ttest_ind(*raw_data, alternative='larger')
print('t-test p (ONE TAILED) = %.3f' % zt[1])

Observations:
 [[4780. 1181.]
 [3243.  796.]]
Chi-sq p = 0.898
Z-test p (TWO TAILED) = 0.898
t-test p (TWO TAILED) = 0.898
Z-test p (ONE TAILED) = 0.551
t-test p (ONE TAILED) = 0.551
